In [40]:
import numpy as np
import pandas as pd # update to 2.0 when they fix the bug
import hvplot.pandas  # noqa
#For Monte Carlo
import random
#Visualization
import holoviews as hv
from tqdm import tqdm

#Historical Data
import yfinance as yf
import time

In [41]:
stocks = ["BA",'NVDA', 'FCNCA','KO', 'PEP', 'AC', 'TSLA', 'AMZN', 
         'NFLX', 'CAT']
#stocks = ["MSFT"]

In [42]:
from datetime import datetime

def convert_date_format(date_string, input_format='%Y/%m/%d', output_format='%Y-%m-%d'):
    date_object = datetime.strptime(date_string, input_format)
    return date_object.strftime(output_format)


In [43]:
#begin_date = "01/01/2014"
begin_date = "2014/01/01"
#end_date = "20/03/2023"
end_date = "2023/01/01"

begin_date = convert_date_format(begin_date)
end_date = convert_date_format(end_date)

In [44]:
def generate_stock_returns(stocks_list, begin_date, end_date):
    prices = pd.DataFrame()
    for stock in stocks_list:
        ticker = yf.Ticker(stock)
        df_ = ticker.history(start=begin_date, end=end_date)['Close']
        df_.rename(stock, inplace=True)
        prices = pd.concat([prices, df_], axis=1)
    prices.index.name = "Date"
    return prices

prices = generate_stock_returns(stocks, begin_date, end_date)

returns = prices.pct_change()

cov = returns.cov()

np.random.seed(42)
weights = np.random.random(len(stocks))
weights /= np.sum(weights)

rp = (returns.mean()*252)@weights
port_var = weights@(cov*252)@weights
sharpe = (rp-rf)/np.sqrt(port_var)

def portfolio_metrics(weights, index='Trial'):
    rp = (returns.mean()*252)@weights 
    port_var = weights@(cov*252)@weights
    sharpe = (rp-rf)/np.sqrt(port_var)
    df = pd.DataFrame({"Expected Return": rp,
                       "Portfolio Variance":port_var,
                       'Portfolio Std': np.sqrt(port_var),
                       'Sharpe Ratio': sharpe}, index=[index])
    return df

portfolios = pd.DataFrame(columns=[*stocks, "Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"])

for i in range(10000):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)
    portfolios.loc[i, stocks] = weights
    metrics = portfolio_metrics(weights,i)
    portfolios.loc[i, ["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]] = \
    metrics.loc[i,["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]]

    
# Replace infinite values with NaN
portfolios = portfolios.replace([np.inf, -np.inf], np.nan)

best_portfolio_index = portfolios["Sharpe Ratio"].idxmax()
worst_portfolio_index = portfolios["Sharpe Ratio"].idxmin()

best_portfolio = portfolios.loc[best_portfolio_index, stocks]
best_portfolio = best_portfolio / best_portfolio.sum() * 100

worst_portfolio = portfolios.loc[worst_portfolio_index, stocks]
worst_portfolio = worst_portfolio / worst_portfolio.sum() * 100


In [45]:
worst_portfolio

BA       24.993637
NVDA      2.688488
FCNCA    21.873078
KO       19.176401
PEP       1.944400
AC       16.399321
TSLA      1.465153
AMZN      1.988831
NFLX      1.580780
CAT       7.889911
Name: 7928, dtype: float64

In [46]:
best_portfolio


BA        1.993245
NVDA     36.399699
FCNCA     2.179239
KO        6.699744
PEP      25.408135
AC        5.825717
TSLA      4.003780
AMZN      5.755241
NFLX      2.319783
CAT       9.415417
Name: 4541, dtype: float64